# Decision Tree Implementation

Your task is to implement parts of the Decision Tree Classification algorithm from scratch (i.e., without importing any libraries or packages for decision trees). A decision tree is a supervised learning model used for classification tasks and is comprised of the following steps:

1. Split the Dataset: Create possible splits for each feature of the dataset. For each split, separate the data into two groups based on the feature's threshold value.

2. Calculate the Split Metric: For each split, calculate an evaluation metric such as Gini Impurity to determine the best split. Gini Impurity measures how often a randomly chosen element would be incorrectly labeled, aiming to create pure groups. A Gini Impurity of 0 represents perfect purity, while higher values indicate more mixed groups.

3. Build the Tree Recursively: Use a recursive function to create nodes, splitting the data until a stopping criterion is met (e.g., maximum depth of tree, minimum number of samples, or no more possible splits).

4. Classify New Data: Use the resulting decision tree to classify a new data point based on the feature values and decision rules at each node.

You will be given a 2D array of float values, train_data, as training data, where each subarray represents a unique sample. The last element in each subarray represents the true class label (e.g., 0 or 1). You will also be given test_data, which you need to classify using the decision tree you build.

In [33]:
train_data = [[2.8, 1.0, 0], 
 [1.3, 3.1, 1], 
 [3.6, 2.7, 0], 
 [2.9, 1.9, 1], 
 [1.5, 0.9, 0], 
 [3.7, 1.5, 1]]


test_data = [[3.0, 1.0], 
 [1.8, 2.5], 
 [2.2, 1.7]]


In [34]:
# skeleton code

# Step 1: Split dataset based on a feature and a threshold
def split_dataset(data: list[list[float]], feature_index: int, threshold: float) -> tuple[list[list[float]], list[list[float]]]:
    """
    Split the dataset into two groups based on a feature index and a threshold value.

    Args:
        data: The dataset to be split.
        feature_index: Index of the feature to split on.
        threshold: Threshold value for splitting.

    Returns:
        A tuple containing two lists, each representing a group of data points.
    """
    left, right = [], []
    for row in data:
        if row[feature_index] < threshold:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Step 2: Calculate Gini Impurity or Information Gain
def calculate_gini(groups: list[list[list[float]]], classes: list[int]) -> float:
    """
    Calculate Gini Impurity for a given dataset split.

    Args:
        groups: A list containing two groups of data points.
        classes: A list of unique class labels.

    Returns:
        Gini Impurity of the split.
    """
    n_samples = sum([len(group) for group in groups])
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        unique_labels = set(row[-1] for row in group)
        for label in unique_labels:
            count = sum(1 for row in group if row[-1] == label)
            p = count / size
            score += p ** 2
        gini += (1.0 - score) * (size / n_samples)
    return gini


# Step 3: Select the best split point
def get_best_split(data: list[list[float]]) -> dict[str, list[int, float, tuple[list[list[float]], list[list[float]]]]]:
    """
    Find the best split point for the dataset.

    Args:
        data: The dataset to find the best split for.

    Returns:
        A dictionary containing the best feature index, threshold, and groups.
    """
    class_values = list(set(row[-1] for row in data))
    best_feature, best_threshold, best_score, best_groups = None, None, float('inf'), None

    for feature_index in range(len(data[0]) - 1):
        unique_values = set(row[feature_index] for row in data)
        for threshold in unique_values:
            groups = split_dataset(data, feature_index, threshold)
            gini = calculate_gini(groups, class_values)
            if gini < best_score:
                best_feature, best_threshold, best_score, best_groups = feature_index, threshold, gini, groups

    return {'feature_index': best_feature, 'threshold': best_threshold, 'groups': best_groups}

# Helper function to create a terminal node
def to_terminal(group: list[list[float]]) -> int:
    """
    Create a terminal node.

    Args:
        group: A list of data points in a group.

    Returns:
        The class label that occurs most frequently in the group.
    """
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Step 4: Build the tree
def build_tree(data: list[list[float]], max_depth: int, min_size: int, depth: int = 0) -> dict[str,list[int, float, dict]]:
    """
    Build the decision tree recursively.

    Args:
        data: The dataset to build the tree from.
        max_depth: The maximum depth of the tree.
        min_size: The minimum number of samples required to split a node.
        depth: The current depth of the tree (used during recursion).

    Returns:
        A dictionary representing the decision tree.
    """
    split = get_best_split(data)
    left_group, right_group = split['groups']

    if not left_group or not right_group:
        return to_terminal(left_group + right_group)

    if depth >= max_depth:
        return {'left': to_terminal(left_group), 'right': to_terminal(right_group)}

    node = {'feature_index': split['feature_index'], 'threshold': split['threshold'], 'left': None, 'right': None}

    if len(left_group) <= min_size:
        node['left'] = to_terminal(left_group)
    else:
        node['left'] = build_tree(left_group, max_depth, min_size, depth + 1)

    if len(right_group) <= min_size:
        node['right'] = to_terminal(right_group)
    else:
        node['right'] = build_tree(right_group, max_depth, min_size, depth + 1)

    return node

# Step 5: Make a prediction
def predict(node: dict[str, list[int, float, dict]], row: list[float]) -> int:
    """
    Make a prediction for a given data point by traversing the decision tree.

    Args:
        node: The decision tree.
        row: The data point to classify.

    Returns:
        The predicted class label.
    """
    if row[node['feature_index']] < node['threshold']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Step 6: Decision Tree Classifier
def decision_tree(train_data: list[list[float]], test_data: list[list[float]], max_depth: int, min_size: int) -> list[int]:
    """
    Build the decision tree and classify the given test data.

    Args:
        train_data: The training dataset.
        test_data: The dataset to classify.
        max_depth: The maximum depth of the tree.
        min_size: The minimum number of samples required to split a node.

    Returns:
        A list of predicted class labels for each sample in test_data.
    """
    tree = build_tree(train_data, max_depth, min_size)
    predictions = [predict(tree, row) for row in test_data]
    return predictions


In [35]:
predictions = decision_tree(train_data, test_data, 100, 10 )